File Preprocessing

In [1]:
import os
import codecs
import spacy
import numpy
import gensim
from spacy import displacy
import pandas as pd
import itertools as it

In [2]:
# PICK THE LANGUAGE OF THE CORPUS YOU WISH TO EXAMINE
language = 'de'

In [3]:
%%time
if language == 'fr':
    print('loading french model')
    nlp = spacy.load("fr_core_news_md")
elif language == 'de':
    print('loading german model')
    nlp = spacy.load("de_core_news_md")   


loading german model
CPU times: user 50.7 s, sys: 9.23 s, total: 59.9 s
Wall time: 1min


We open the preprocessed de file

In [4]:
directory = "../corpus_samples/OCRd/GBE/processed/"
filename = directory + language +'.txt'

In [5]:
with codecs.open(filename, encoding='utf_8') as f:
  lines = f.readlines()
  full_text = '\n'.join(lines)
print(full_text)

veroiientlichung Königl Preußische Geodätische Institut FOLGE Nr.69 Jahresbericht Direktor Königliche Geodätische Institut April April Potsdam drucken Stankiewiez Buchdruckerei G.m.b Berlin Exzellenz Königliche Staatsminister Minister geistlich Unterrichts-Angelegenheiten Herrn Trott Solz gehorsamst überreichen Jahresbericht Direktor Königliche Geodätische Institut April April sächlichen ausgeben belaufen Verwendung folgen stellen Tagegelder Reisekosten Stations- beobachtungen außerhalb entfallen Herrn Flotows Aufenthalt Amerika Beobachtung verbunden ausgeben außerordentlich Rechenarbeiten Schreibhilfe verschieden Reise Verwaltung Dotationsfonds Heizmaterial Heizen Reinigen Diensträume Druckkosten dergl Buch Zeitschrift dergl Postgeld dergl Schreibmaterial Instandhaltung Abänderung Anschaffung Untersuchung Instrument aus- wärtige Mechaniker und so weiter mechanische Werkstatt photo- graphische Kammer einschließlich Gehilfenlöhne Material verschieden Mobiliarbeschaffungen insgemein wiss

In [6]:
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence

In [7]:
intermediate_directory = '../corpus_samples/lemmatized/' + language + '/'

def line_page(filename):
    """
    generator function to read in pages from the file
    and un-escape the original line breaks in the text
    """
    
    with codecs.open(filename, encoding='utf_8') as f:
        for page in f:
            yield page.replace('\\r', '\r')

def lemmatized_sentence_corpus(filename):
    """
    generator function to use spaCy to parse pages,
    lemmatize the text, and yield sentences
    """
    
    for parsed_page in nlp.pipe(line_page(filename),
                                  batch_size=10000, n_threads=4):
        
        for sent in parsed_page.sents:
            yield u' '.join([token.lemma_ for token in sent])


unigram_sentences_filepath = os.path.join(intermediate_directory,
                                          'unigram_sentences_all.txt')

with codecs.open(unigram_sentences_filepath, 'w', encoding='utf_8') as f:
    for sentence in lemmatized_sentence_corpus(filename):
        f.write(sentence + '\n')

In [8]:
unigram_sentences = LineSentence(unigram_sentences_filepath)

In [9]:
for unigram_sentence in it.islice(unigram_sentences, 0, 240):
    print(u' '.join(unigram_sentence))
    print(u'')

veroiientlichung Königl

preußisch Geodätische Institut

FOLGE Nr.69

Jahresbericht Direktor

königlich Geodätische Institut April April Potsdam drucken Stankiewiez

Buchdruckerei G.m.b

Berlin

Exzellenz

königlich Staatsminister

Minister geistlich Unterrichts-Angelegenheiten Herrn Trott Solz gehorsamst überreichen Jahresbericht

Direktor

königlich Geodätische Institut April April sächlichen ausgeben belaufen

Verwendung folgen stellen

Tagegelder Reisekosten

Stations- beobachtungen außerhalb entfallen Herrn Flotows Aufenthalt Amerika Beobachtung verbinden ausgeben außerordentlich Rechenarbeiten

Schreibhilfe verscheiden reisen

Verwaltung Dotationsfonds Heizmaterial Heizen

Reinigen Diensträume Druckkosten dergl buchen

Zeitschrift dergl Postgeld

dergl Schreibmaterial Instandhaltung Abänderung

Anschaffung Untersuchung Instrument aus- wärtige Mechaniker und so weit mechanische Werkstatt photo- graphische Kammer einschließlich Gehilfenlöhne Material verscheiden Mobiliarbeschaffung

In [10]:
bigram_model_filepath = os.path.join(intermediate_directory, 'bigram_model')

In [11]:
bigram_model = Phrases(unigram_sentences)

bigram_model.save(bigram_model_filepath)
    
# load the finished model from disk
bigram_model = Phrases.load(bigram_model_filepath)

In [12]:
bigram_sentences_filepath = os.path.join(intermediate_directory,
                                         'bigram_sentences_all.txt')
with codecs.open(bigram_sentences_filepath, 'w', encoding='utf_8') as f:
    for unigram_sentence in unigram_sentences:
        bigram_sentence = u' '.join(bigram_model[unigram_sentence])
        f.write(bigram_sentence + '\n')

In [13]:
bigram_sentences = LineSentence(bigram_sentences_filepath)

In [14]:
for bigram_sentence in it.islice(bigram_sentences, 0, 240):
    print(u' '.join(bigram_sentence))
    print(u'')

veroiientlichung Königl

preußisch_Geodätische Institut

FOLGE Nr.69

Jahresbericht_Direktor

königlich_Geodätische Institut_April April_Potsdam drucken_Stankiewiez

Buchdruckerei_G.m.b

Berlin

Exzellenz

königlich_Staatsminister

Minister_geistlich Unterrichts-Angelegenheiten_Herrn Trott_Solz gehorsamst_überreichen Jahresbericht

Direktor

königlich_Geodätische Institut_April April_sächlichen ausgeben_belaufen

Verwendung_folgen stellen

Tagegelder_Reisekosten

Stations-_beobachtungen außerhalb_entfallen Herrn_Flotows Aufenthalt Amerika Beobachtung_verbinden ausgeben_außerordentlich Rechenarbeiten

Schreibhilfe_verscheiden reisen

Verwaltung_Dotationsfonds Heizmaterial_Heizen

Reinigen_Diensträume Druckkosten_dergl buchen

Zeitschrift_dergl Postgeld

dergl_Schreibmaterial Instandhaltung_Abänderung

Anschaffung_Untersuchung Instrument_aus- wärtige_Mechaniker und_so weit_mechanische Werkstatt_photo- graphische_Kammer einschließlich_Gehilfenlöhne Material_verscheiden Mobiliarbeschaffung

In [15]:
trigram_model_filepath = os.path.join(intermediate_directory,
                                      'trigram_model_all')

trigram_model = Phrases(bigram_sentences)

trigram_model.save(trigram_model_filepath)
    
# load the finished model from disk
trigram_model = Phrases.load(trigram_model_filepath)

trigram_sentences_filepath = os.path.join(intermediate_directory,
                                          'trigram_sentences_all.txt')


with codecs.open(trigram_sentences_filepath, 'w', encoding='utf_8') as f:
        
    for bigram_sentence in bigram_sentences:
            
        trigram_sentence = u' '.join(trigram_model[bigram_sentence])
            
        f.write(trigram_sentence + '\n')

In [16]:
from gensim.corpora import Dictionary, MmCorpus
from gensim.models.ldamulticore import LdaMulticore

import pyLDAvis
import pyLDAvis.gensim
import warnings
import pickle

In [17]:
trigram_dictionary_filepath = os.path.join(intermediate_directory,
                                           'trigram_dict_all.dict')

In [18]:
#extreme filters hyperparams
no_below = 8
no_above = 0.4

trigram_pages = LineSentence(trigram_sentences_filepath)

# learn the dictionary by iterating over all of the pages
trigram_dictionary = Dictionary(trigram_pages)
    
# filter tokens that are very rare or too common from
# the dictionary (filter_extremes) and reassign integer ids (compactify)
trigram_dictionary.filter_extremes(no_below, no_above)
trigram_dictionary.compactify()

trigram_dictionary.save(trigram_dictionary_filepath)
    
# load the finished dictionary from disk
trigram_dictionary = Dictionary.load(trigram_dictionary_filepath)

In [19]:
trigram_bow_filepath = os.path.join(intermediate_directory,
                                    'trigram_bow_corpus_all.mm')

In [20]:
def trigram_bow_generator(filepath):
    """
    generator function to read pages from a file
    and yield a bag-of-words representation
    """
    
    for page in LineSentence(filepath):
        yield trigram_dictionary.doc2bow(page)

In [21]:
MmCorpus.serialize(trigram_bow_filepath,
                       trigram_bow_generator(trigram_sentences_filepath))
    
# load the finished bag-of-words corpus from disk
trigram_bow_corpus = MmCorpus(trigram_bow_filepath)

In [22]:
lda_model_filepath = os.path.join(intermediate_directory, 'lda_model_all')

In [23]:
%%time

with warnings.catch_warnings():
    warnings.simplefilter('ignore')
        
    # workers => sets the parallelism, and should be
        # set to your number of physical cores minus one
    lda = LdaMulticore(trigram_bow_corpus,
                           num_topics=30,
                           id2word=trigram_dictionary,
                           workers=3)
    
lda.save(lda_model_filepath)
    
# load the finished LDA model from disk
lda = LdaMulticore.load(lda_model_filepath)

CPU times: user 760 ms, sys: 630 ms, total: 1.39 s
Wall time: 1.51 s


In [24]:
def explore_topic(topic_number, topn=25):
    """
    accept a user-supplied topic number and
    print out a formatted list of the top terms
    """
        
    print(u'{:20} {}'.format(u'term', u'frequency') + u'\n')

    for term, frequency in lda.show_topic(topic_number, topn=25):
        print(u'{:20} {:.3f}'.format(term, round(frequency, 3)))

In [25]:
explore_topic(topic_number=8)

term                 frequency

ausfahren            0.020
finden               0.019
einigen              0.017
Längenbestimmung     0.012
Herrn                0.012
Station              0.012
werten               0.011
Institut             0.011
definitiv            0.009
Beobachtung          0.009
folgen               0.009
Seite                0.008
hoffen               0.007
Nähe                 0.007
beschäftigen         0.007
Ausgleichung         0.007
zeigen               0.007
Arbeit               0.006
Instrument           0.006
glasen               0.006
Horta                0.006
München              0.006
Längenunterschied    0.006
Potsdam              0.005
Rechnung             0.005


In [26]:
LDAvis_data_filepath = os.path.join(intermediate_directory, 'ldavis_prepared')

In [27]:
%%time
LDAvis_prepared = pyLDAvis.gensim.prepare(lda, trigram_bow_corpus,
                                              trigram_dictionary)

CPU times: user 7.71 s, sys: 250 ms, total: 7.96 s
Wall time: 49.2 s


/opt/conda/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [28]:
pyLDAvis.display(LDAvis_prepared)